In [1]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName('market campaign').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/21 20:24:06 WARN Utils: Your hostname, Pulastyas-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.40 instead (on interface en0)
26/01/21 20:24:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/21 20:24:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
spark.conf.set("spark.sql.shuffle.partitions", "1")

In [10]:
mc = spark.read.options(header=True, inferSchema=True)\
    .csv('data/input/marketing_campaign.csv').dropDuplicates(['user_id', 'created_at', 'product_id'])\
    .orderBy(F.col('user_id'), F.col('created_at'))\
    .coalesce(1)
    
mc.show(truncate=False)

+-------+----------+----------+--------+-----+
|user_id|created_at|product_id|quantity|price|
+-------+----------+----------+--------+-----+
|10     |2019-01-01|101       |3       |55   |
|10     |2019-01-02|119       |5       |29   |
|10     |2019-03-31|111       |2       |149  |
|11     |2019-01-02|105       |3       |234  |
|11     |2019-03-31|120       |3       |99   |
|12     |2019-01-02|112       |2       |200  |
|12     |2019-03-31|110       |2       |299  |
|13     |2019-01-05|113       |1       |67   |
|13     |2019-03-31|118       |3       |35   |
|14     |2019-01-06|109       |5       |199  |
|14     |2019-01-06|107       |2       |27   |
|14     |2019-03-31|112       |3       |200  |
|15     |2019-01-08|105       |4       |234  |
|15     |2019-01-09|110       |4       |299  |
|15     |2019-03-31|116       |2       |499  |
|16     |2019-01-10|113       |2       |67   |
|16     |2019-03-31|107       |4       |27   |
|17     |2019-01-11|116       |2       |499  |
|17     |2019

In [11]:
mc.explain(True)

== Parsed Logical Plan ==
Repartition 1, false
+- Sort [user_id#177 ASC NULLS FIRST, created_at#178 ASC NULLS FIRST], true
   +- Deduplicate [user_id#177, created_at#178, product_id#179]
      +- Relation [user_id#177,created_at#178,product_id#179,quantity#180,price#181] csv

== Analyzed Logical Plan ==
user_id: int, created_at: date, product_id: int, quantity: int, price: int
Repartition 1, false
+- Sort [user_id#177 ASC NULLS FIRST, created_at#178 ASC NULLS FIRST], true
   +- Deduplicate [user_id#177, created_at#178, product_id#179]
      +- Relation [user_id#177,created_at#178,product_id#179,quantity#180,price#181] csv

== Optimized Logical Plan ==
Repartition 1, false
+- Sort [user_id#177 ASC NULLS FIRST, created_at#178 ASC NULLS FIRST], true
   +- Aggregate [user_id#177, created_at#178, product_id#179], [user_id#177, created_at#178, product_id#179, first(quantity#180, false) AS quantity#233, first(price#181, false) AS price#235]
      +- Relation [user_id#177,created_at#178,produc

In [ ]:
spark.stop()